In [63]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
from bs4 import BeautifulSoup as bs

wikipedia_link='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
raw_wikipedia_page= requests.get(wikipedia_link).text

soup = BeautifulSoup(raw_wikipedia_page,'lxml')

In [64]:
soup

<!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="en">
<head>
<meta charset="utf-8"/>
<title>List of postal codes of Canada: M - Wikipedia</title>
<script>document.documentElement.className = document.documentElement.className.replace( /(^|\s)client-nojs(\s|$)/, "$1client-js$2" );</script>
<script>(window.RLQ=window.RLQ||[]).push(function(){mw.config.set({"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":false,"wgNamespaceNumber":0,"wgPageName":"List_of_postal_codes_of_Canada:_M","wgTitle":"List of postal codes of Canada: M","wgCurRevisionId":867606113,"wgRevisionId":867606113,"wgArticleId":539066,"wgIsArticle":true,"wgIsRedirect":false,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Communications in Ontario","Postal codes in Canada","Toronto","Ontario-related lists"],"wgBreakFrames":false,"wgPageContentLanguage":"en","wgPageContentModel":"wikitext","wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","w

In [66]:
#import geocoder
table = soup.find('table')

Postcode      = []
Borough       = []
Neighbourhood = []

# print(table)

for tr_cell in table.find_all('tr'):
    
    counter = 1
    Postcode_var      = -1
    Borough_var       = -1
    Neighbourhood_var = -1
    
    for td_cell in tr_cell.find_all('td'):
        if counter == 1: 
            Postcode_var = td_cell.text
        if counter == 2: 
            Borough_var = td_cell.text
            tag_a_Borough = td_cell.find('a')
            
        if counter == 3: 
            Neighbourhood_var = str(td_cell.text).strip()
            tag_a_Neighbourhood = td_cell.find('a')
            
        counter +=1
        
    if (Postcode_var == 'Not assigned' or Borough_var == 'Not assigned' or Neighbourhood_var == 'Not assigned'): 
        continue
    try:
        if ((tag_a_Borough is None) or (tag_a_Neighbourhood is None)):
            continue
    except:
        pass
    if(Postcode_var == -1 or Borough_var == -1 or Neighbourhood_var == -1):
        continue
        
    Postcode.append(Postcode_var)
    Borough.append(Borough_var)
    Neighbourhood.append(Neighbourhood_var)

In [67]:
unique_p = set(Postcode)
# print('num of unique Postal codes:', len(unique_p))
Postcode_u      = []
Borough_u       = []
Neighbourhood_u = []


for postcode_unique_element in unique_p:
    p_var = ''; b_var = ''; n_var = ''; 
    for postcode_idx, postcode_element in enumerate(Postcode):
        if postcode_unique_element == postcode_element:
            p_var = postcode_element;
            b_var = Borough[postcode_idx]
            if n_var == '': 
                n_var = Neighbourhood[postcode_idx]
            else:
                n_var = n_var + ', ' + Neighbourhood[postcode_idx]
    Postcode_u.append(p_var)
    Borough_u.append(b_var)
    Neighbourhood_u.append(n_var)
Postcode_u

['M3C',
 'M9L',
 'M9W',
 'M1X',
 'M1B',
 'M1V',
 'M5H',
 'M3J',
 'M5C',
 'M6K',
 'M8Y',
 'M2J',
 'M5A',
 'M9M',
 'M3L',
 'M4L',
 'M1N',
 'M4N',
 'M4K',
 'M6R',
 'M4X',
 'M9N',
 'M9P',
 'M9A',
 'M5K',
 'M4G',
 'M5M',
 'M4Y',
 'M1R',
 'M4C',
 'M6P',
 'M6A',
 'M1M',
 'M1H',
 'M4V',
 'M5X',
 'M1E',
 'M2L',
 'M1S',
 'M5T',
 'M3K',
 'M1J',
 'M4A',
 'M5S',
 'M5E',
 'M1L',
 'M2H',
 'M3A',
 'M1P',
 'M4B',
 'M1G',
 'M6B',
 'M6L',
 'M5P',
 'M5L',
 'M6S',
 'M3H',
 'M8Z',
 'M1C',
 'M9C',
 'M4W',
 'M4T',
 'M4H',
 'M5B',
 'M6J',
 'M2M',
 'M4E',
 'M1T',
 'M5J',
 'M2K',
 'M9V',
 'M8X',
 'M4J',
 'M8V',
 'M6H',
 'M6M',
 'M1W',
 'M1K',
 'M5V',
 'M9B',
 'M8W',
 'M9R',
 'M2R',
 'M5R']

In [68]:
toronto_dict = {'Postal Code':Postcode_u, 'Borough':Borough_u, 'Neighbourhood':Neighbourhood_u}
df_toronto = pd.DataFrame.from_dict(toronto_dict)
df_toronto.to_csv('toronto_part1.csv')
df_toronto.head()


,Postal Code,Borough,Neighbourhood
0,M3C,North York,Flemingdon Park
1,M9L,North York,Humber Summit
2,M9W,Etobicoke,Northwest
3,M1X,Scarborough,Upper Rouge
4,M1B,Scarborough,"Rouge, Malvern"


In [69]:
df_toronto.shape

(84, 3)

In [80]:
postcode = df_toronto.set_index('Postal Code')


In [81]:
df_geo = pd.read_csv('Geospatial_Coordinates.csv')
df_geo = df_geo.set_index('Postal Code')
df_geo

,Latitude,Longitude
Postal Code,,
M1B,43.806686,-79.194353
M1C,43.784535,-79.160497
M1E,43.763573,-79.188711
M1G,43.770992,-79.216917
M1H,43.773136,-79.239476
M1J,43.744734,-79.239476
M1K,43.727929,-79.262029
M1L,43.711112,-79.284577
M1M,43.716316,-79.239476


In [101]:

df_tor = pd.concat([postcode, df_geo], axis=1, join_axes=[postcode.index])

df_tor = df_tor.reset_index()


In [107]:
from geopy.geocoders import Nominatim

geolocator = Nominatim(user_agent="test", format_string="%s, Ontario, CA")
location = geolocator.geocode("Toronto")
print(location.latitude, location.longitude)

43.653963 -79.387207


In [103]:

import requests # library to handle requests

# !conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')



Libraries imported.


In [108]:
toronto_latitude = location.latitude;
toronto_latitude

43.653963

In [113]:
# for the city Toronto, latitude and longtitude are manually extracted via google search
toronto_latitude = location.latitude; 
toronto_longitude = location.longitude
map_toronto = folium.Map(location = [toronto_latitude, toronto_longitude], zoom_start = 10.7)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_tor['Latitude'], df_tor['Longitude'], df_tor['Borough'], df_tor['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='green',
        fill=True,
        fill_color='white',
        fill_opacity=0.7).add_to(map_toronto)  
    

map_toronto

In [114]:
CLIENT_ID = 'DMM1FMUTQTWJ4OCB1300IPRTQDHQCITA41S4GGURWKBSW2YO' # your Foursquare ID
CLIENT_SECRET = 'DFPPZRGSBT5D0JWTEEJ2GB2MJASUKCCC3NN5PXASVFG0KQYA' # your Foursquare Secret
VERSION = '20190102' # Foursquare API version

In [116]:
from geopy.geocoders import Nominatim

geolocator = Nominatim(user_agent="test", format_string="%s,Toronto, Ontario, CA")
location1 = geolocator.geocode("North York")
print(location1.latitude, location1.longitude)

43.7709163 -79.4124102


In [117]:
address = 'North York'
latitude_N = location1.latitude
longitude_N = location1.longitude
print('The geograpical coordinate of North York are {}, {}.'.format(latitude_N, longitude_N))


The geograpical coordinate of North York are 43.7709163, -79.4124102.


In [121]:
N_data = df_tor[df_tor['Borough'] == 'North York']
N_data



,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3C,North York,Flemingdon Park,43.725900,-79.340923
1,M9L,North York,Humber Summit,43.756303,-79.565963
7,M3J,North York,"Northwood Park, York University",43.767980,-79.487262
11,M2J,North York,Henry Farm,43.778517,-79.346556
13,M9M,North York,"Emery, Humberlea",43.724766,-79.532242
14,M3L,North York,Downsview West,43.739015,-79.506944
26,M5M,North York,Bedford Park,43.733283,-79.419750
31,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763
37,M2L,North York,"Silver Hills, York Mills",43.757490,-79.374714
40,M3K,North York,CFB Toronto,43.737473,-79.464763


In [124]:
map_N = folium.Map(location=[latitude_N, longitude_N], zoom_start=10)

# add markers to map
for lat, lng, label in zip(N_data['Latitude'], N_data['Longitude'], N_data['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='red',
        fill=True,
        fill_color='white',
        fill_opacity=0.7).add_to(map_N)  
    
map_N

In [125]:
neighborhood_latitude = N_data.loc[0, 'Latitude'] # neighbourhood latitude value
neighborhood_longitude = N_data.loc[0, 'Longitude'] # neighbourhood longitude value

neighborhood_name = N_data.loc[0, 'Neighbourhood'] # neighbourhood name

print('Latitude and longitude values of "{}" are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of "Flemingdon Park" are 43.72589970000001, -79.340923.


In [127]:
LIMIT = 100
radius = 500
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude_N, longitude_N, VERSION, radius, LIMIT)

In [128]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5c2c653edb04f53ab91903fe'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Willowdale',
  'headerFullLocation': 'Willowdale, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 51,
  'suggestedBounds': {'ne': {'lat': 43.775416304500006,
    'lng': -79.40619010427393},
   'sw': {'lat': 43.7664162955, 'lng': -79.41863029572606}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '563d44fccd1044ad67a744fb',
       'name': "The Captain's Boil",
       'location': {'address': '5313 Yonge St',
        'lat': 43.773255217045026,
        'lng': -79.41380541792645,
        'labeledLatLngs': [{'label': 'display',
          'lat': 43.7

In [129]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [131]:
import json # library to handle JSON files
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

venues = results['response']['groups'][0]['items']  
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head(10)

,name,categories,lat,lng
0,The Captain's Boil,Seafood Restaurant,43.773255,-79.413805
1,Aroma Espresso Bar,Café,43.769449,-79.413081
2,Loblaws,Grocery Store,43.768648,-79.412597
3,Burrito Boyz,Burrito Place,43.773054,-79.414082
4,Starbucks,Coffee Shop,43.768192,-79.413021
5,Cineplex Cinemas Empress Walk,Movie Theater,43.768625,-79.412613
6,Dakgogi,Korean Restaurant,43.773010,-79.413875
7,The Keg,Steakhouse,43.766579,-79.412131
8,Owl of Minerva,Korean Restaurant,43.773328,-79.413915
9,Konjiki Ramen,Ramen Restaurant,43.766998,-79.412222


In [132]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

51 venues were returned by Foursquare.


In [133]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [134]:
N_venues = getNearbyVenues(names=N_data['Neighbourhood'],
                                   latitudes=N_data['Latitude'],
                                   longitudes=N_data['Longitude']
                                  )

Flemingdon Park
Humber Summit
Northwood Park, York University
Henry Farm
Emery, Humberlea
Downsview West
Bedford Park
Lawrence Heights, Lawrence Manor
Silver Hills, York Mills
CFB Toronto
Victoria Village
Hillcrest Village
Parkwoods
Glencairn
Maple Leaf Park
Bathurst Manor, Wilson Heights
Newtonbrook, Willowdale
Bayview Village
Willowdale West


In [135]:
N_venues.head(10)

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Flemingdon Park,43.7259,-79.340923,Fitness Connection,43.727473,-79.341707,Gym
1,Flemingdon Park,43.7259,-79.340923,Sorento Restaurant,43.726575,-79.341989,Italian Restaurant
2,Flemingdon Park,43.7259,-79.340923,Tilley Endurables,43.726863,-79.342614,Clothing Store
3,Flemingdon Park,43.7259,-79.340923,Memories of Japan,43.727347,-79.341364,Japanese Restaurant
4,Flemingdon Park,43.7259,-79.340923,Swiss Chalet Rotisserie & Grill,43.726737,-79.341403,Restaurant
5,Flemingdon Park,43.7259,-79.340923,Japanese Canadian Cultural Centre,43.726429,-79.334971,General Entertainment
6,Flemingdon Park,43.7259,-79.340923,Skiis and Biikes,43.726351,-79.342977,Bike Shop
7,Flemingdon Park,43.7259,-79.340923,The Beer Store,43.726987,-79.341494,Beer Store
8,Flemingdon Park,43.7259,-79.340923,Genghis Khan Mongolian Grill,43.726906,-79.341216,Asian Restaurant
9,Flemingdon Park,43.7259,-79.340923,Tim Hortons,43.722897,-79.339117,Coffee Shop


In [136]:

N_venues.tail(10)

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
175,"Bathurst Manor, Wilson Heights",43.754328,-79.442259,Sunrise Foods & Catering,43.755178,-79.441669,Grocery Store
176,"Newtonbrook, Willowdale",43.789053,-79.408493,Wedgewood park,43.790635,-79.405494,Park
177,Bayview Village,43.786947,-79.385975,Sun Star Chinese Cuisine 翠景小炒,43.787914,-79.381234,Chinese Restaurant
178,Bayview Village,43.786947,-79.385975,Maxim's Cafe and Patisserie,43.787863,-79.380751,Café
179,Bayview Village,43.786947,-79.385975,TD Canada Trust,43.788074,-79.380367,Bank
180,Bayview Village,43.786947,-79.385975,Kaga Sushi,43.787758,-79.381090,Japanese Restaurant
181,Willowdale West,43.782736,-79.442259,Tim Hortons,43.780940,-79.444231,Coffee Shop
182,Willowdale West,43.782736,-79.442259,Shoppers Drug Mart,43.783245,-79.446349,Pharmacy
183,Willowdale West,43.782736,-79.442259,Tov-Li,43.784214,-79.446098,Pizza Place
184,Willowdale West,43.782736,-79.442259,Hartman's,43.784312,-79.446213,Butcher


In [137]:

N_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Bathurst Manor, Wilson Heights",17,17,17,17,17,17
Bayview Village,4,4,4,4,4,4
Bedford Park,26,26,26,26,26,26
CFB Toronto,3,3,3,3,3,3
Downsview West,5,5,5,5,5,5
"Emery, Humberlea",1,1,1,1,1,1
Flemingdon Park,21,21,21,21,21,21
Glencairn,5,5,5,5,5,5
Henry Farm,62,62,62,62,62,62


In [138]:
print('There are {} uniques categories.'.format(len(N_venues['Venue Category'].unique())))

There are 91 uniques categories.


In [139]:
N_onehot = pd.get_dummies(N_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
N_onehot['Neighborhood'] = N_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [N_onehot.columns[-1]] + list(N_onehot.columns[:-1])
N_onehot = N_onehot[fixed_columns]

N_onehot.head()

,Neighborhood,Accessories Store,Airport,American Restaurant,Asian Restaurant,Bakery,Bank,Bar,Baseball Field,Basketball Court,...,Tailor Shop,Tea Room,Thai Restaurant,Theater,Toy / Game Store,Video Game Store,Video Store,Vietnamese Restaurant,Wings Joint,Women's Store
0,Flemingdon Park,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Flemingdon Park,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Flemingdon Park,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Flemingdon Park,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Flemingdon Park,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [140]:
N_onehot.shape

(185, 92)

In [150]:
N_grouped = N_onehot.groupby('Neighborhood').mean().reset_index()
N_grouped.head(7)

,Neighborhood,Accessories Store,Airport,American Restaurant,Asian Restaurant,Bakery,Bank,Bar,Baseball Field,Basketball Court,...,Tailor Shop,Tea Room,Thai Restaurant,Theater,Toy / Game Store,Video Game Store,Video Store,Vietnamese Restaurant,Wings Joint,Women's Store
0,"Bathurst Manor, Wilson Heights",0.0,0.000000,0.000000,0.000000,0.0,0.058824,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.058824,0.0,0.0,0.0
1,Bayview Village,0.0,0.000000,0.000000,0.000000,0.0,0.250000,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
2,Bedford Park,0.0,0.000000,0.038462,0.000000,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.038462,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
3,CFB Toronto,0.0,0.333333,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
4,Downsview West,0.0,0.000000,0.000000,0.000000,0.0,0.200000,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
5,"Emery, Humberlea",0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,1.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
6,Flemingdon Park,0.0,0.000000,0.000000,0.095238,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0


In [151]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [152]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = N_grouped['Neighborhood']

for ind in np.arange(N_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(N_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Bathurst Manor, Wilson Heights",Coffee Shop,Grocery Store,Shopping Mall,Frozen Yogurt Shop,Fried Chicken Joint,Fast Food Restaurant,Diner,Deli / Bodega,Pizza Place,Restaurant
1,Bayview Village,Japanese Restaurant,Chinese Restaurant,Café,Bank,Event Space,Deli / Bodega,Department Store,Dim Sum Restaurant,Diner,Dog Run
2,Bedford Park,Italian Restaurant,Fast Food Restaurant,Coffee Shop,Grocery Store,Liquor Store,Pizza Place,Pharmacy,Butcher,Comfort Food Restaurant,Japanese Restaurant
3,CFB Toronto,Park,Airport,Bus Stop,Empanada Restaurant,Construction & Landscaping,Cosmetics Shop,Deli / Bodega,Department Store,Dim Sum Restaurant,Diner
4,Downsview West,Grocery Store,Bank,Shopping Mall,Hotel,Moving Target,Fried Chicken Joint,General Entertainment,Construction & Landscaping,Cosmetics Shop,Deli / Bodega
5,"Emery, Humberlea",Baseball Field,Women's Store,Event Space,Construction & Landscaping,Cosmetics Shop,Deli / Bodega,Department Store,Dim Sum Restaurant,Diner,Dog Run
6,Flemingdon Park,Gym,Asian Restaurant,Coffee Shop,Beer Store,Grocery Store,Sandwich Place,Fast Food Restaurant,Italian Restaurant,Japanese Restaurant,Dim Sum Restaurant
7,Glencairn,Japanese Restaurant,Pizza Place,Pub,Metro Station,Park,Women's Store,Comfort Food Restaurant,Construction & Landscaping,Cosmetics Shop,Deli / Bodega
8,Henry Farm,Clothing Store,Fast Food Restaurant,Coffee Shop,Food Court,Toy / Game Store,Tea Room,Bakery,Restaurant,Japanese Restaurant,Metro Station
9,Hillcrest Village,Golf Course,Pool,Mediterranean Restaurant,Dog Run,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Cosmetics Shop,Deli / Bodega,Department Store


In [155]:
from sklearn.cluster import KMeans

#N_data = N_data.drop(16)
# set number of clusters
kclusters = 5

N_grouped_clustering = N_grouped.drop('Neighborhood', 1)


# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(N_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_

array([0, 0, 0, 4, 0, 1, 0, 0, 0, 0, 3, 0, 0, 2, 0, 4, 0, 0])

In [162]:
N_data = N_data.reset_index().drop(18)

In [163]:
N_merged = N_data

# add clustering labels
N_merged['Cluster Labels'] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
N_merged = N_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighbourhood')

N_merged

,index,Postal Code,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,0,M3C,North York,Flemingdon Park,43.725900,-79.340923,0,Gym,Asian Restaurant,Coffee Shop,Beer Store,Grocery Store,Sandwich Place,Fast Food Restaurant,Italian Restaurant,Japanese Restaurant,Dim Sum Restaurant
1,1,M9L,North York,Humber Summit,43.756303,-79.565963,0,Pizza Place,Empanada Restaurant,Comfort Food Restaurant,Construction & Landscaping,Cosmetics Shop,Deli / Bodega,Department Store,Dim Sum Restaurant,Diner,Dog Run
2,7,M3J,North York,"Northwood Park, York University",43.767980,-79.487262,0,Coffee Shop,Furniture / Home Store,Massage Studio,Bar,Falafel Restaurant,Frozen Yogurt Shop,Fried Chicken Joint,Cosmetics Shop,Deli / Bodega,Department Store
3,11,M2J,North York,Henry Farm,43.778517,-79.346556,4,Clothing Store,Fast Food Restaurant,Coffee Shop,Food Court,Toy / Game Store,Tea Room,Bakery,Restaurant,Japanese Restaurant,Metro Station
4,13,M9M,North York,"Emery, Humberlea",43.724766,-79.532242,0,Baseball Field,Women's Store,Event Space,Construction & Landscaping,Cosmetics Shop,Deli / Bodega,Department Store,Dim Sum Restaurant,Diner,Dog Run
5,14,M3L,North York,Downsview West,43.739015,-79.506944,1,Grocery Store,Bank,Shopping Mall,Hotel,Moving Target,Fried Chicken Joint,General Entertainment,Construction & Landscaping,Cosmetics Shop,Deli / Bodega
6,26,M5M,North York,Bedford Park,43.733283,-79.419750,0,Italian Restaurant,Fast Food Restaurant,Coffee Shop,Grocery Store,Liquor Store,Pizza Place,Pharmacy,Butcher,Comfort Food Restaurant,Japanese Restaurant
7,31,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763,0,Clothing Store,Boutique,Furniture / Home Store,Event Space,Miscellaneous Shop,Coffee Shop,Accessories Store,Gift Shop,Vietnamese Restaurant,Airport
8,37,M2L,North York,"Silver Hills, York Mills",43.757490,-79.374714,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,40,M3K,North York,CFB Toronto,43.737473,-79.464763,0,Park,Airport,Bus Stop,Empanada Restaurant,Construction & Landscaping,Cosmetics Shop,Deli / Bodega,Department Store,Dim Sum Restaurant,Diner


In [167]:
import matplotlib.cm as cm
import matplotlib.colors as colors

# create map
map_clusters = folium.Map(location = [latitude_N, longitude_N], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(N_merged['Latitude'], N_merged['Longitude'], N_merged['Neighbourhood'], N_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters